# Phase 6: Deployment

## Fannie Mae 2008Q1 Stress Testing - Credit Default Risk Modeling

---

### CRISP-DM Phase 6: Generate Reports and Deploy Model

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
from sklearn.metrics import roc_curve
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries imported successfully!")
print(f"Report generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 6.1 Load Evaluation Results

In [ ]:
# Load evaluation data
with open('phase5_evaluation.pkl', 'rb') as f:
    eval_data = pickle.load(f)

comparison_df = eval_data['comparison_df']
best_model_name = eval_data['best_model_name']
best_auc = eval_data['best_auc']
cm = eval_data['confusion_matrix']
feat_imp_df = eval_data['feature_importance_df']
results = eval_data['results']
y_test = eval_data['y_test']
features = eval_data['features']

print(f"Best Model: {best_model_name}")
print(f"Best AUC-ROC: {best_auc:.4f}")

## 6.2 Create Final Dashboard

In [ ]:
# Create comprehensive dashboard
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# 1. ROC Curves
ax1 = axes[0, 0]
colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']
for i, (model_name, res) in enumerate(results.items()):
    fpr, tpr, _ = roc_curve(y_test, res['y_pred_proba'])
    ax1.plot(fpr, tpr, label=f"{model_name} (AUC={res['auc_roc']:.3f})",
            linewidth=2, color=colors[i % len(colors)])
ax1.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax1.axhline(y=0.7, color='orange', linestyle=':', alpha=0.7, label='Target AUC=0.70')
ax1.set_xlabel('False Positive Rate', fontsize=11)
ax1.set_ylabel('True Positive Rate', fontsize=11)
ax1.set_title('ROC Curves Comparison', fontsize=13, fontweight='bold')
ax1.legend(loc='lower right', fontsize=9)
ax1.grid(True, alpha=0.3)

# 2. Model Metrics Comparison
ax2 = axes[0, 1]
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
x = np.arange(len(metrics))
width = 0.2
for i, (model_name, res) in enumerate(results.items()):
    values = [res['accuracy'], res['precision'], res['recall'], res['f1_score'], res['auc_roc']]
    ax2.bar(x + i*width, values, width, label=model_name, alpha=0.8, color=colors[i])
ax2.axhline(y=0.70, color='red', linestyle='--', alpha=0.7, label='Target (0.70)')
ax2.set_xlabel('Metrics', fontsize=11)
ax2.set_ylabel('Score', fontsize=11)
ax2.set_title('Model Performance Comparison', fontsize=13, fontweight='bold')
ax2.set_xticks(x + width * 1.5)
ax2.set_xticklabels(metrics, rotation=45)
ax2.legend(loc='upper left', fontsize=8)
ax2.set_ylim(0, 1.1)
ax2.grid(True, alpha=0.3, axis='y')

# 3. Confusion Matrix
ax3 = axes[1, 0]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax3,
            xticklabels=['No Default', 'Default'],
            yticklabels=['No Default', 'Default'],
            annot_kws={'size': 14})
ax3.set_xlabel('Predicted', fontsize=11)
ax3.set_ylabel('Actual', fontsize=11)
ax3.set_title(f'Confusion Matrix - {best_model_name}', fontsize=13, fontweight='bold')

# 4. Feature Importance
ax4 = axes[1, 1]
if feat_imp_df is not None:
    if 'Importance' in feat_imp_df.columns:
        plot_df = feat_imp_df.nlargest(10, 'Importance')
        ax4.barh(plot_df['Feature'], plot_df['Importance'], color='steelblue')
    else:
        plot_df = feat_imp_df.nlargest(10, 'Abs_Importance')
        ax4.barh(plot_df['Feature'], plot_df['Abs_Importance'], color='steelblue')
    ax4.set_xlabel('Importance', fontsize=11)
    ax4.set_title(f'Top 10 Features - {best_model_name}', fontsize=13, fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='x')

plt.suptitle('Fannie Mae 2008Q1 Stress Testing - Credit Default Prediction', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('Final_Dashboard.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Dashboard saved to Final_Dashboard.png")

## 6.3 Generate Final Report

In [ ]:
# Get best model results
best_res = results[best_model_name]

# Feature importance string
if feat_imp_df is not None:
    if 'Importance' in feat_imp_df.columns:
        top_features = feat_imp_df.nlargest(5, 'Importance')[['Feature', 'Importance']].to_string(index=False)
    else:
        top_features = feat_imp_df.nlargest(5, 'Abs_Importance')[['Feature', 'Coefficient']].to_string(index=False)
else:
    top_features = 'N/A'

# Generate report
report = f"""
================================================================================
FANNIE MAE 2008Q1 STRESS TESTING - CREDIT DEFAULT PREDICTION
CRISP-DM ANALYSIS FINAL REPORT
================================================================================

Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

================================================================================
1. EXECUTIVE SUMMARY
================================================================================

This analysis applied the CRISP-DM methodology to predict mortgage loan defaults
during Q1 2008, a critical period of the financial crisis.

KEY RESULTS:
• Best Model: {best_model_name}
• AUC-ROC: {best_auc:.4f}
• Target Achievement: {"✓ ACHIEVED" if best_auc >= 0.70 else "✗ Below target (0.70)"}

================================================================================
2. DATA OVERVIEW
================================================================================

• Period: Q1 2008 (Financial Crisis Stress Period)
• Source: Fannie Mae Single-Family Loan Performance Data
• Sample Size: 1,000,000 records
• Features: {len(features)} engineered features
• Target: is_default (binary: 0=No Default, 1=Default)

================================================================================
3. MODEL PERFORMANCE
================================================================================

{comparison_df.round(4).to_string()}

BEST MODEL: {best_model_name}
------------------------------------
• AUC-ROC:   {best_res['auc_roc']:.4f}
• Accuracy:  {best_res['accuracy']:.4f}
• Precision: {best_res['precision']:.4f}
• Recall:    {best_res['recall']:.4f}
• F1-Score:  {best_res['f1_score']:.4f}

================================================================================
4. CONFUSION MATRIX
================================================================================

True Negatives (Correct No-Default):  {cm[0,0]:,}
False Positives (False Alarms):       {cm[0,1]:,}
False Negatives (Missed Defaults):    {cm[1,0]:,}
True Positives (Correct Defaults):    {cm[1,1]:,}

================================================================================
5. KEY RISK DRIVERS
================================================================================

Top 5 Features by Importance:
{top_features}

================================================================================
6. RECOMMENDATIONS
================================================================================

1. RISK MONITORING:
   - Monitor loans with high deferred UPB ratios
   - Flag loans with accumulated delinquent interest
   - Track high LTV loans (>80%) closely

2. EARLY WARNING SYSTEM:
   - Implement monthly scoring for all active loans
   - Set threshold alerts for high-risk predictions
   - Create intervention workflows for flagged loans

3. MODEL IMPROVEMENT (if AUC < 0.70):
   - Add Acquisition file features (FICO, original LTV, DTI)
   - Include geographic risk factors (state, MSA)
   - Engineer time-series features (payment history)
   - Consider XGBoost or LightGBM for better performance

================================================================================
7. FILES GENERATED
================================================================================

• Phase1_Business_Understanding.ipynb
• Phase2_Data_Understanding.ipynb
• Phase3_Data_Preparation.ipynb
• Phase4_Modeling.ipynb
• Phase5_Evaluation.ipynb
• Phase6_Deployment.ipynb
• Final_Dashboard.png
• Final_Report.txt

================================================================================
END OF REPORT
================================================================================
"""

# Display report
print(report)

# Save report
with open('Final_Report.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✓ Report saved to Final_Report.txt")

## 6.4 Save Model Comparison

In [ ]:
# Save comparison to CSV
comparison_df.to_csv('Model_Comparison.csv')
print("✓ Model comparison saved to Model_Comparison.csv")

# Display final comparison
print("\nFinal Model Comparison:")
display(comparison_df.round(4).style.highlight_max(axis=0, color='lightgreen'))

---
# ✅ CRISP-DM Analysis Complete!

## Summary

This project successfully implemented the complete CRISP-DM methodology for credit
default prediction during the 2008 financial crisis stress period.

### Phases Completed:
1. ✅ **Business Understanding** - Defined objectives and success criteria
2. ✅ **Data Understanding** - Explored and analyzed Fannie Mae data
3. ✅ **Data Preparation** - Cleaned data and engineered features
4. ✅ **Modeling** - Trained 4 classification models
5. ✅ **Evaluation** - Compared performance and analyzed results
6. ✅ **Deployment** - Generated reports and visualizations

### Deliverables:
- 6 Jupyter Notebooks (one per CRISP-DM phase)
- Final Dashboard visualization
- Comprehensive text report
- Model comparison CSV

---
*Analysis completed using Python, scikit-learn, and the CRISP-DM methodology.*